In [2]:
import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sympy 

In [5]:
# path = "../data/tst_cal_005.csv"
path = "../test/tst_data/data7_tst_cal_004_onArmArbitraryMotions.csv"
df = pd.read_csv(path)
df.head(0)

,IMU_TIMESTAMP(0)_0,IMU_GYROSCOPE(0)_0,IMU_GYROSCOPE(0)_1,IMU_GYROSCOPE(0)_2,IMU_GYROSCOPE(1)_0,IMU_GYROSCOPE(1)_1,IMU_GYROSCOPE(1)_2,IMU_ACCELEROMETER(0)_0,IMU_ACCELEROMETER(0)_1,IMU_ACCELEROMETER(0)_2,...,IMU_LINEAR_ACCELERATION(1)_1,IMU_LINEAR_ACCELERATION(1)_2,IMU_QUATERNION(0)_0,IMU_QUATERNION(0)_1,IMU_QUATERNION(0)_2,IMU_QUATERNION(0)_3,IMU_QUATERNION(1)_0,IMU_QUATERNION(1)_1,IMU_QUATERNION(1)_2,IMU_QUATERNION(1)_3


In [6]:
for i,row in df.iterrows():
    print( f"{i}: {row['IMU_TIMESTAMP(0)_0']}" )
    omega1 = np.array([ row['IMU_GYROSCOPE(0)_0'], row['IMU_GYROSCOPE(0)_1'], row['IMU_GYROSCOPE(0)_2']] )
    print(omega1)
# omega1 = np.array(df[])


0: 186.36
[ 0.461426 -0.578729  0.091809]
1: 186.38
[-0.074598  0.018682  0.589053]
2: 186.46
[-0.008116 -0.530423  0.628629]
3: 186.48
[ 0.507567 -0.240286  0.926681]
4: 186.5
[-0.036588 -0.590702  0.859564]
5: 186.52
[ 0.139763 -0.420197  0.547559]
6: 186.54
[-0.085062 -0.291085  0.74424 ]
7: 186.56
[-0.399665 -0.321838  0.893094]
8: 186.58
[ 0.041942 -0.320583  0.585371]
9: 186.6
[-0.497981 -0.052176  0.707976]
10: 186.62
[-0.673069  0.356822  0.750184]
11: 186.64
[ 0.015141 -0.051359  0.820796]
12: 186.66
[0.692744 0.040426 0.352786]
13: 186.68
[-0.421363  0.678201 -0.264183]
14: 186.7
[-1.345118  0.865798  0.007737]
15: 186.72
[-1.570439  1.164959 -0.018511]
16: 186.74
[-1.585081  0.926101 -0.614797]
17: 186.76
[-1.130606  0.9267   -0.422264]
18: 186.78
[-0.951581  0.86669  -0.100172]
19: 186.8
[-0.973851  0.787256 -0.459087]
20: 186.82
[-1.968794  1.005206 -0.5782  ]
21: 186.84
[-1.801061  1.095444 -0.618812]
22: 186.86
[-1.356049  1.015965 -0.294944]
23: 186.88
[-1.66877  0.6049